In [26]:
from dataclasses import dataclass, field
from marshmallow import Schema, fields, post_load, post_dump

In [27]:
from uuid import uuid4

def make_uuid(prefix):
    def _make_uuid():
        return '-'.join([prefix, str(uuid4())])
    return _make_uuid

In [28]:
@dataclass(order=True)
class Todo:
    user_id: int = field()
    _id: int = field(compare=False, repr=False)
    uuid: str = field()
    title: str = field()
    completed: bool = field(compare=False)
        

In [29]:
class TodoSchema(Schema):
    user_id = fields.Integer()
    _id = fields.Integer(missing=-1)
    uuid = fields.String(missing=make_uuid('TODO'))
    title = fields.String()
    completed = fields.Boolean(missing=False)
    
    @post_load
    def make_todo(self, data, **kwargs):
        return Todo(**data)
    
    @post_dump
    def maybe_remove_id(self, data, **kwargs):
        if data['_id']==-1:
            data.pop('_id')
        return data

In [30]:
t1 = TodoSchema().load({
    'user_id': 1,
    'title': 'Lorem ipsum',
})

t2 = TodoSchema().load({
    'user_id': 0,
    'title': 'Lorem ipsum',
    'completed': True
})

t3 = TodoSchema().load({
    'user_id': 0,
    'title': 'Lorem ipsum dolor sit amet'
})

In [31]:
dumped  = TodoSchema().dump(t1)

In [32]:
dumped

{'user_id': 1,
 'title': 'Lorem ipsum',
 'uuid': 'TODO-275eaff5-b2b4-41a6-a743-29be1a8474c8',
 'completed': False}

In [33]:
t1==t2

False

In [34]:
todos = [t1,t2,t3]
todos.sort()
todos

[Todo(user_id=0, uuid='TODO-171788ee-dc40-4d9c-8c1b-558c5fe1d52e', title='Lorem ipsum dolor sit amet', completed=False),
 Todo(user_id=0, uuid='TODO-b3e6b755-c5c2-41ea-8bdb-dfc3d889961e', title='Lorem ipsum', completed=True),
 Todo(user_id=1, uuid='TODO-275eaff5-b2b4-41a6-a743-29be1a8474c8', title='Lorem ipsum', completed=False)]

In [35]:
todos_list = TodoSchema(many=True).dump(todos)
todos_list

[{'user_id': 0,
  'title': 'Lorem ipsum dolor sit amet',
  'uuid': 'TODO-171788ee-dc40-4d9c-8c1b-558c5fe1d52e',
  'completed': False},
 {'user_id': 0,
  'title': 'Lorem ipsum',
  'uuid': 'TODO-b3e6b755-c5c2-41ea-8bdb-dfc3d889961e',
  'completed': True},
 {'user_id': 1,
  'title': 'Lorem ipsum',
  'uuid': 'TODO-275eaff5-b2b4-41a6-a743-29be1a8474c8',
  'completed': False}]

In [36]:
TodoSchema(many=True).load(todos_list)

[Todo(user_id=0, uuid='TODO-171788ee-dc40-4d9c-8c1b-558c5fe1d52e', title='Lorem ipsum dolor sit amet', completed=False),
 Todo(user_id=0, uuid='TODO-b3e6b755-c5c2-41ea-8bdb-dfc3d889961e', title='Lorem ipsum', completed=True),
 Todo(user_id=1, uuid='TODO-275eaff5-b2b4-41a6-a743-29be1a8474c8', title='Lorem ipsum', completed=False)]

In [37]:
import json

class MockDao:
    
    def __init__(self, path, schema):
        self._path = path
        self._schema = schema
        try:
            with open(self._path,'r') as f:
                self._items=self._schema.load(json.load(f))
        except:
            self._items = []
    
    def get_all(self):
        return self._items
    
    def get_by_uuid(self, uuid):
        item = [t for t in self._items if t.uuid==uuid]
        return item[0] if item else None
    
    def add_item(self, item):
        self._items.append(item)
        self._replace_file()
        
    def delete_by_uuid(self, uuid):
        item = [t for t in self._items if t.uuid==uuid]
        if item:
            self._items.pop(self._items.index(item[0]))
        self._replace_file()
            
    def update_item_by_uuid(self, uuid, data):
        item = self.get_by_uuid(uuid)
        for k,v in data.items():
            setattr(item,k,v)
        self._replace_file()
        
    def _replace_file(self):
        with open(self._path, 'w') as f:
            serialized = self._schema.dump(self._items)
            json.dump(serialized,f)
        
    def clear(self):
        self._items = []
        self._replace_file()

In [51]:
TODOS_DB = "/todos/database/todos_db.json"
dao = MockDao(TODOS_DB, TodoSchema(many=True))
dao.clear()

In [52]:
dao.get_all()

[]

In [53]:
todos

[Todo(user_id=0, uuid='TODO-171788ee-dc40-4d9c-8c1b-558c5fe1d52e', title='Lorem ipsum dolor sit amet', completed=False),
 Todo(user_id=0, uuid='TODO-b3e6b755-c5c2-41ea-8bdb-dfc3d889961e', title='Lorem ipsum', completed=True),
 Todo(user_id=1, uuid='TODO-275eaff5-b2b4-41a6-a743-29be1a8474c8', title='Lorem ipsum', completed=False)]

In [54]:
schema = TodoSchema(many=True)
copied_todos = schema.load(schema.dump(todos))
for t in copied_todos:
    dao.add_item(t) 

In [55]:
dao._items[2] is todos[2]

False

In [56]:
dao._items[2] == todos[2]

True

In [57]:
dao.get_all()

[Todo(user_id=0, uuid='TODO-171788ee-dc40-4d9c-8c1b-558c5fe1d52e', title='Lorem ipsum dolor sit amet', completed=False),
 Todo(user_id=0, uuid='TODO-b3e6b755-c5c2-41ea-8bdb-dfc3d889961e', title='Lorem ipsum', completed=True),
 Todo(user_id=1, uuid='TODO-275eaff5-b2b4-41a6-a743-29be1a8474c8', title='Lorem ipsum', completed=False)]

In [58]:
dao.get_by_uuid(todos[2].uuid)

Todo(user_id=1, uuid='TODO-275eaff5-b2b4-41a6-a743-29be1a8474c8', title='Lorem ipsum', completed=False)

In [59]:
dao.delete_by_uuid(todos[0].uuid)

In [62]:
dao.get_all()

[Todo(user_id=0, uuid='TODO-b3e6b755-c5c2-41ea-8bdb-dfc3d889961e', title='Lorem ipsum', completed=True),
 Todo(user_id=0, uuid='TODO-275eaff5-b2b4-41a6-a743-29be1a8474c8', title='Lorem ipsum', completed=False)]

In [61]:
dao.update_item_by_uuid(todos[2].uuid, {'user_id':0})

In [ ]:
dao.get_all()

In [ ]:
todos

In [82]:
class MyClass:
    
    def __init__(self, visible, hidden):
        self.visible = visible
        self.__hidden = hidden
        
    def get_hidden(self):
        return self.__hidden
    
class MyOtherClass(MyClass):
    def __init__(self, visible, hidden):
        super().__init__(visible, hidden)
        self.__hidden = 3

In [83]:
b = MyOtherClass(10,100)

In [85]:
b.get_hidden()

100

In [86]:
dir(b)

['_MyClass__hidden',
 '_MyOtherClass__hidden',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'get_hidden',
 'visible']

In [72]:
a = MyClass(10,100)

In [73]:
a.get_hidden()

100

In [76]:
a._MyClass__hidden

100